In [0]:
"""
Script de traitement de scénarios de défauts simulés pour la maintenance prédictive.

Ce script parcourt différents sous-dossiers contenant des fichiers CSV issus de simulations
de défauts sur des microturbines. Pour chaque fichier :
- Il identifie le début du défaut à partir du nom du fichier (via des expressions régulières).
- Il tronque les données à partir de ce point pour ne garder que la phase de dégradation.
- Il calcule la Remaining Useful Life (RUL) comme la différence entre le temps final
  et chaque timestamp.
- Il ne conserve que les colonnes de capteurs communes à tous les fichiers.
- Il ajoute un identifiant d’unité (`unit_id`) basé sur le nom du fichier.

Tous les fichiers valides sont fusionnés en un unique DataFrame et exportés en CSV,
prêt à être utilisé pour des tâches de Machine Learning en régression de RUL.

Destination finale :
    /Volumes/dbe_dbx_internships/justin/predictive_maintenance/all_scenarios_with_RUL.csv
    
"""


In [0]:
import os
import pandas as pd
import numpy as np
import glob
import re

base_folder = "/Volumes/dbe_dbx_internships/justin/predictive_maintenance/" # Dossier qui reprend tout les dossiers de chaque défaut
output_file = os.path.join(base_folder, "all_scenarios_with_RUL.csv") 

subfolders = [ # Sous-dossiers contenus dans le base_folder
    "Defaut_CC_T",
    "Defaut_Compresseur",
    "Defaut_Fuite_Fuel",
    "Defaut_Noise_TOT",
    "Defaut_Rec_T",
    "Defaut_Recuperateur",
    "Defaut_Turbine",
    "FiltreAir",
    "Defaut_Mix_Final",
    "Defaut_Tuyaux_cc_t",
    "Defaut_Tuyaux_comp_rec",
    "Defaut_Tuyaux_rec_cc",
    "Defaut_Tuyaux_turb_rec" 
]

common_cols = [ # Colonnes communes à l'ensemble des dossiers de défauts
    "Time","Nref","N","Power_Shaft","Eff_Cycle","Power_Comp","Power_Turb","Beta_Comp","Beta_Turb",
    "Is_Eff_Comp","Is_Eff_Turb","T_amb","T_c_in","T_c_out","T_rec_c_in","T_rec_c_out","T_cc_in",
    "T_cc_out","T_t_in","T_t_out","T_rec_h_in","T_rec_h_out","p_amb","p_c_in","p_c_out",
    "p_rec_c_in","p_rec_c_out","p_cc_in","p_cc_out","p_t_in","p_t_out","p_rec_h_in","p_rec_h_out",
    "m_amb","m_c_in","m_c_out","m_rec_c_in","m_rec_c_out","m_cc_in","m_cc_out",
    "m_t_in","m_t_out","m_rec_h_in","m_rec_h_out","m_fuel"
]

fault_mapping = { # Mapping pour la lecture des fichiers de défauts -> booléen : début du défaut
    "NoiseTOT": "StartTOT",
    "NoiseFuel": "StartFuel",
    "bool_eta_comp_deg": "start_comp_eff",
    "bool_eta_turb": "start",
    "bool_foul_CC_turb": "start_foul_CC_turb",
    "bool_foul_recup_turb": "start_foul_T_REC"
}

def extract_active_start(file_path):
    """
    Extrait le moment de début du premier défaut actif à partir du chemin du fichier.

    Cette version utilise le nom de dossier parent pour détecter le dossier 'Defaut_Mix' et
    applique toujours le pattern SXXX_timestamp sur le basename.
    """
    filename = os.path.basename(file_path)
    folder = os.path.basename(os.path.dirname(file_path))
    
    matches = re.findall(r"(S[A-Z0-9]+)_(\d+)", filename) # Trouver tous les tags S..._(num) qui indique début du défaut
    starts = [int(val) for tag, val in matches if int(val) != 0]

    if 'Defaut_Mix' in folder: # Cas défaut mix
        return min(starts) if starts else None

    for foul_pattern in ['SF', 'SFR']: # Cas Recuperateur et Filtre à air
        foul_match = re.search(f"{foul_pattern}_?(\d+)", filename)
        if foul_match and int(foul_match.group(1)) != 0:
            starts.append(int(foul_match.group(1)))

    for bool_key, start_key in fault_mapping.items():# Autres cas 
        if re.search(f"{bool_key}_(1)", filename):
            m = re.search(f"{start_key}_?(\d+)", filename)
            if m:
                starts.append(int(m.group(1)))

    fuite_bool = re.search(r"boolFuites_(\d)_(\d)_(\d)_(\d)", filename) # Cas avec les fuites
    fuite_start = re.search(r"startFuites_(\d+)_(\d+)_(\d+)_(\d+)", filename)
    if fuite_bool and fuite_start:
        bools = list(map(int, fuite_bool.groups()))
        vals = list(map(int, fuite_start.groups()))
        starts += [v for b, v in zip(bools, vals) if b == 1]

    return min(starts) if starts else None


def apply_rul(file):
    
    """
    Charge un fichier CSV simulé et applique l’étiquetage RUL à partir du début du défaut.

    - Charge le fichier CSV contenant les données capteurs d'une unité simulée.
    - Détecte le début du défaut (via `extract_active_start`).
    - Coupe le dataset à partir de ce moment.
    - Calcule la RUL (Remaining Useful Life) comme la distance au temps final.
    - Ne conserve que les colonnes standardisées (définies dans `common_cols`).
    - Assigne un identifiant `unit_id` basé sur le nom de fichier.

    Parameters
    ----------
    file : str
        Chemin vers un fichier CSV de simulation.

    Returns
    -------
    pd.DataFrame or None
        Un dataframe avec les colonnes capteurs, la RUL, et l’`unit_id`,
        ou None si le fichier ne contient pas de défaut exploitable.
    """
    
    try:
        df = pd.read_csv(file)
    except Exception as e:
        print(f"Erreur lecture fichier {file} : {e}")
        return None

    if "Time" not in df.columns:
        print(f"Colonne 'Time' absente dans {file}")
        return None

    start = extract_active_start(os.path.basename(file))
    if start is None:
        print(f"Aucun défaut actif détecté dans : {file}")
        return None

    df = df[df["Time"] >= start].copy()
    end_time = df["Time"].max()
    df["RUL"] = (end_time - df["Time"]).round(2)  


    df["unit_id"] = os.path.basename(file)

    missing_cols = [col for col in common_cols if col not in df.columns] # Ne garder que les colonnes communes
    if missing_cols:
        print(f"[!] Fichier {file} ignoré (colonnes manquantes: {missing_cols})")
        return None

    return df[common_cols + ["RUL", "unit_id"]]

rul_dfs = []
for sub in subfolders:
    data_folder = os.path.join(base_folder, sub)
    files = glob.glob(os.path.join(data_folder, "*.csv"))
    print(f"\n[INFO] Traitement de {sub} ({len(files)} fichiers au total)")

    files_selected = []
    for f in files:
        fname = os.path.basename(f)

        match = re.search(r"Sim_(\d+)_", fname) # Cas "classique"
        if not match:
            match = re.search(r"Sim_[a-zA-Z_]+_(\d+)_", fname) # Cas "tuyaux" → Sim_cc_t_xxx_ ou autre

        if match:
            sim_num = int(match.group(1))
            if 1 <= sim_num <= 80:
                files_selected.append(f)

    print(f"[INFO] → {len(files_selected)} fichiers sélectionnés pour {sub} (Sim 1 à 80)")

    for f in files_selected:
        labeled = apply_rul(f)
        if labeled is not None:
            rul_dfs.append(labeled)

if not rul_dfs:
    print("\n[NON] Aucun fichier exploitable dans tous les dossiers.")
else:
    df_final = pd.concat(rul_dfs, ignore_index=True)
    df_final.to_csv(output_file, index=False)
    print(f"\n[OK] Export total vers : {output_file} ({df_final.shape[0]} lignes)")
